In [25]:
def loaddataBinance_FromTo(symbol, from_date, to_date, timeframe):
    import pandas as pd
    import ccxt
    from datetime import datetime, timedelta

    # Khởi tạo kết nối đến sàn Binance
    exchange = ccxt.binance()

    # Định dạng ngày tháng
    since = exchange.parse8601(from_date + 'T00:00:00Z')
    end = exchange.parse8601(to_date + 'T00:00:00Z')

    # Lấy dữ liệu thị trường từ sàn Binance
    # 1d
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=since, limit=1000)

    # Chuyển dữ liệu thành DataFrame
    data = pd.DataFrame(ohlcv, columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
    data['Timestamp'] = pd.to_datetime(data['Timestamp'], unit='ms')         
    data = data.rename(columns={'Timestamp': 'Datetime'})

    return data

In [26]:
import pandas as pd
import plotly.graph_objects as go
import redis
import numpy as np
from datetime import datetime, timedelta

# ##############################################Step 0: Các tham số để lấy dữ liệu###############################
symbol = 'ETHUSDT'
from_date = (datetime.now() - timedelta(days=0)).strftime('%Y-%m-%d')  # Lấy ngày hôm qua
to_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
timeframe = '1m' # 1m

# ##############################################Step 1: Lấy dữ liệu##############################################
data = loaddataBinance_FromTo(symbol, from_date, to_date, timeframe)

# ##############################################Step 2: Chiến lược##############################################  
# Đặt thời gian cho MA và ATR
ma_fast_period = 50
ma_slow_period = 200
atr_period = 14
atr_threshold = 9 # Xác định ngưỡng biến động thị trường

data['MA_Fast'] = data['Close'].rolling(window=ma_fast_period).mean()
data['MA_Slow'] = data['Close'].rolling(window=ma_slow_period).mean()

# Tính True Range (TR)
data['prev_close'] = data['Close'].shift(1)  # Lấy giá đóng cửa của ngày trước đó
data['high_low'] = data['High'] - data['Low']
data['high_close'] = abs(data['High'] - data['prev_close'])
data['low_close'] = abs(data['Low'] - data['prev_close'])
data['TR'] = data[['high_low', 'high_close', 'low_close']].max(axis=1)

# Tính ATR
data['ATR'] = data['TR'].rolling(window=atr_period).mean()

# Xác định tín hiệu mua
data['Buy_Signal'] = (data['MA_Fast'] >= data['MA_Slow']) & (data['ATR'] <= atr_threshold)

# Xác định tín hiệu bán
data['Sell_Signal'] = (data['MA_Fast'] < data['MA_Slow']) & (data['ATR'] <= atr_threshold)

In [27]:
data.to_csv('Buoi3_4 Chienluoc.csv')
data

,Datetime,Open,High,Low,Close,Volume,MA_Fast,MA_Slow,prev_close,high_low,high_close,low_close,TR,ATR,Buy_Signal,Sell_Signal
0,2024-01-04 00:00:00,2209.72,2209.72,2208.33,2208.80,334.9229,NaN,NaN,NaN,1.39,NaN,NaN,1.39,NaN,False,False
1,2024-01-04 00:01:00,2208.80,2209.71,2207.43,2208.39,866.8891,NaN,NaN,2208.80,2.28,0.91,1.37,2.28,NaN,False,False
2,2024-01-04 00:02:00,2208.39,2209.56,2207.75,2209.56,328.6582,NaN,NaN,2208.39,1.81,1.17,0.64,1.81,NaN,False,False
3,2024-01-04 00:03:00,2209.56,2209.56,2207.23,2207.25,300.6349,NaN,NaN,2209.56,2.33,0.00,2.33,2.33,NaN,False,False
4,2024-01-04 00:04:00,2207.25,2207.88,2206.44,2207.87,246.0093,NaN,NaN,2207.25,1.44,0.63,0.81,1.44,NaN,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2024-01-04 16:35:00,2262.02,2262.02,2260.81,2261.75,302.3482,2269.8250,2255.26370,2262.02,1.21,0.00,1.21,1.21,2.493571,True,False
996,2024-01-04 16:36:00,2261.74,2262.34,2260.82,2262.23,494.2660,2269.6886,2255.40010,2261.75,1.52,0.59,0.93,1.52,2.462143,True,False
997,2024-01-04 16:37:00,2262.23,2265.00,2262.22,2263.83,1347.6526,2269.5828,2255.54830,2262.23,2.78,2.77,0.01,2.78,2.342857,True,False
998,2024-01-04 16:38:00,2263.82,2265.37,2262.37,2264.58,355.8971,2269.4842,2255.70025,2263.83,3.00,1.54,1.46,3.00,2.434286,True,False


In [28]:
import plotly.graph_objects as go
import plotly.subplots as sp  # Sử dụng plotly.subplots để tạo nhiều biểu đồ
import pandas as pd 

# Trực quan hóa dữ liệu (tùy chọn)
fig = go.Figure()

# Tạo subplot với 2 hàng
fig = sp.make_subplots(rows=2, cols=1, shared_xaxes=True, 
                       subplot_titles=('Price and Moving Averages', 'Average True Range (ATR)'))

# Thêm đường giá đóng cửa
fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', name='Close Price'), row=1, col=1)

# Thêm đường MA_Fast và MA_Slow vào biểu đồ hàng đầu tiên
fig.add_trace(go.Scatter(x=data.index, y=data['MA_Fast'], mode='lines', name='MA Fast'), row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data['MA_Slow'], mode='lines', name='MA Slow'), row=1, col=1)

# Đánh dấu các tín hiệu mua và bán
fig.add_trace(go.Scatter(x=data[data['Buy_Signal'] == 1].index, 
                         y=data[data['Buy_Signal'] == 1]['Close'], 
                         mode='markers', name='Buy Signal', marker=dict(color='green', size=10)), row=1, col=1)
fig.add_trace(go.Scatter(x=data[data['Sell_Signal'] == 1].index, 
                         y=data[data['Sell_Signal'] == 1]['Close'], 
                         mode='markers', name='Sell Signal', marker=dict(color='red', size=10)), row=1, col=1)

# Thêm đường ATR vào biểu đồ hàng thứ hai
fig.add_trace(go.Scatter(x=data.index, y=data['ATR'], mode='lines', name='ATR'), row=2, col=1)

# Cấu hình và hiển thị biểu đồ
fig.update_layout(title='Moving Average Crossover and ATR Signals', xaxis_title='Date', yaxis_title='Price', 
    height=800  # Tăng kích thước cao của biểu đồ lên 800
)

fig.update_xaxes(title_text='Date', row=2, col=1)
fig.update_yaxes(title_text='ATR', row=2, col=1)
fig.show()